# Text Summariser with Sentiment Analysis

## Importing Libraries

In [ ]:
import gradio as gr
from articles import *
from transformers import pipeline


## Summarizer

In [ ]:

def summarise(text, max_length=130, min_length=30, model='facebook/bart-large-cnn'):
    '''
    This function takes in a text and returns a summary of the text along with the sentiment of the summary.
    '''
    
    summarizer = pipeline("summarization", model=model) # Summarizer pipeline
    
    result = summarizer(text, max_length=max_length, min_length=min_length) # Summarize the text
    
    sent_analysis = pipeline(
        "sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis") # Sentiment analysis pipeline
    
    sent = sent_analysis(result[0]['summary_text'])[0] # Get the sentiment of the summary
    
    if sent['label'] == 'POS': # If the sentiment is positive
        sent1 = 'Sentiment of this Article is Positive with a score of {:.2f}%'.format(sent['score']*100)
        
    elif sent['label'] == 'NEU': # If the sentiment is neutral
        sent1 = 'Sentiment of this Article is Neutral with a score of {:.2f}%'.format(
            sent['score']*100)
        
    elif sent['label'] == 'NEG': # If the sentiment is negative
        sent1 = 'Sentiment of this Article is Negative with a score of {:.2f}%'.format(
            sent['score']*100)
        
    return result[0]['summary_text'], sent1 # Return the summary and the sentiment


## Making Gradio App

In [ ]:

demo = gr.Blocks() # Create a gradio block

with demo:
    gr.Markdown("# Text Summarizer with Sentiment Analysis")
    gr.Markdown("### Enter the text you want to summarise or choose from examples below")
    with gr.Tabs():
        with gr.TabItem("Examples"): # If the user wants to use the examples
            with gr.Column():
                rad = gr.components.Radio(
                    ['Article 1', 'Article 2', 'Article 3'], label='Select article and wait till it appears!') # Radio button to select the article
                text1 = gr.Textbox(label='Example') # Textbox to show the article
                rad2 = gr.components.Radio(
                    ['Facebook\'s Large Bart Model', 'DistilBart', 'Google\'s Pegasus'], label='Select Model for summarisation') # Radio button to select the model
                max1 = gr.components.Slider(100, 200, 130, label='Max Length for Summary') # Slider to select the max length of the summary
                min1 = gr.components.Slider(20, 100, 30, label='Min Length for Summary') # Sliders to select the min length of the summary
            submit1 = gr.Button('Submit')
        with gr.TabItem("Do it yourself!"): # If the user wants to enter their own text
            with gr.Column():
                text2 = gr.components.Textbox(label='Enter your own text here!')
                rad3 = gr.components.Radio(
                    ['Facebook\'s Large Bart Model', 'DistilBart', 'Google\'s Pegasus'], label='Select Model for summarisation') # Radio button to select the model
                max2 = gr.components.Slider(0, 200, 130, label='Max Length for Summary') # Slider to select the max length of the summary
                min2 = gr.components.Slider(0, 100, 30, label='Min Length for Summary') # Sliders to select the min length of the summary
            submit2 = gr.Button('Submit')
                
        def action1(choice): # Function to show the article when the user selects the article
            if choice == 'Article 1':
                return ARTICLE_1
            elif choice == 'Article 2':
                return ARTICLE_2
            elif choice == 'Article 3':
                return ARTICLE_3

        def models(model_name): # Function to select the model
            if model_name == 'Facebook\'s Large Bart Model':
                return 'facebook/bart-large-cnn'
            elif model_name == 'DistilBart':
                return 'sshleifer/distilbart-cnn-12-6'
            elif model_name == 'Google\'s Pegasus':
                return 'google/pegasus-xsum'
            elif model_name is None:
                return 'facebook/bart-large-cnn'
        
        rad.change(action1, rad, text1) # Change the article when the user selects the article
        
        op = gr.outputs.Textbox(label='Summary') # Output for the summary
        op2 = gr.outputs.Textbox(label='Sentiment Analysis of Summary') # Output for the sentiment analysis of the summary
        
        def fn(text, model, max_length, min_length): # Main function
            model = models(model)
            result = summarise(text, max_length, min_length, model)
            return result
        
        submit1.click(fn=fn, inputs=[text1, rad2, max1, min1], outputs=[op, op2]) # Submit button for the examples
        submit2.click(fn=fn, inputs=[text2, rad3, max2, min2], outputs=[op, op2]) # Submit button for the user input
        


## Launching App

In [ ]:
demo.launch(server_port=8080, server_name='0.0.0.0') # Launch the gradio block